In [1]:
from world_machine.train.scheduler import UniformScheduler, ChoiceScheduler
from world_machine.train.stages import StateSaveMethod

from world_machine_experiments.toy1d import Dimensions

In [2]:
n_epoch = 100

n_variation = 0

for n_segment in [1,2]:
    fast_forward_choices = []
    if n_segment >1:
        fast_forward_choices = [True, False]

    for fast_forward in fast_forward_choices:
        for stable_state_epochs in [1, 2, 5]:
            for check_input_masks in [True, False]:
                for state_save_method in [StateSaveMethod.MEAN, StateSaveMethod.REPLACE]:
                    for mask_sensorial_data in [None, 0.1, UniformScheduler(0, 1, n_epoch), ChoiceScheduler([0,1], n_epoch), 0.5]:
                        for short_time_recall in [{}, {Dimensions.NEXT_MEASUREMENT}]:
                        
                            recall_n_past_choices = []
                            recall_stride_choices = []
                            if len(short_time_recall ) > 0:                            
                                recall_n_past_choices = [0, 1, 5]
                                recall_stride_choices = [1,3]
                            
                            for recall_stride in recall_stride_choices:
                                for recall_n_past in recall_n_past_choices:
                                    
                                    recall_n_future_choices = []
                                    if len(short_time_recall) > 0 and recall_n_past > 0:
                                        recall_n_future_choices = [0, 1, 5]
                                    elif len(short_time_recall) > 0:
                                        recall_n_future_choices = [1,5]
                                    
                                    for recall_n_future in recall_n_future_choices:
                                        for positional_encoder_type in ["alibi", "sine"]:
                                            for sensorial_train_losses in [[], [Dimensions.NEXT_MEASUREMENT]]:
                                                for block_configuration in [[Dimensions.NEXT_MEASUREMENT, Dimensions.NEXT_MEASUREMENT],
                                                                            [Dimensions.NEXT_MEASUREMENT, Dimensions.STATE_INPUT],
                                                                            [Dimensions.STATE_INPUT, Dimensions.NEXT_MEASUREMENT]]:
                                                    for state_activation in ["tanh", "ltanh", None]:
                                                        state_regularizer = None
                                                        if state_activation is None:
                                                            state_regularizer = "mse"

                                                        n_variation += 1

In [67]:
n_epoch = 100

n_variation = 0
configurations = {}

for n_segment in [1,2]:
    fast_forward_choices = [False]
    if n_segment >1:
        fast_forward_choices = [True, False]

    for fast_forward in fast_forward_choices:
            for check_input_masks in [True, False]:
                for state_save_method in [StateSaveMethod.MEAN, StateSaveMethod.REPLACE]:
                    for mask_sensorial_data in [UniformScheduler(0, 1, n_epoch)]:
                        for short_time_recall in [(), (Dimensions.NEXT_MEASUREMENT,)]:
                        
                            recall_n_past_choices = [0]
                            recall_stride_choices = [0]
                            if len(short_time_recall ) > 0:                            
                                recall_n_past_choices = [0, 1, 5]
                                recall_stride_choices = [1,3]
                            
                            for recall_stride in recall_stride_choices:
                                for recall_n_past in recall_n_past_choices:
                                    
                                    recall_n_future_choices = []
                                    if len(short_time_recall) > 0 and recall_n_past > 0:
                                        recall_n_future_choices = [0, 1, 5]
                                    elif len(short_time_recall) > 0:
                                        recall_n_future_choices = [1,5]
                                    
                                    for recall_n_future in recall_n_future_choices:
                                                for block_configuration in [(Dimensions.NEXT_MEASUREMENT, Dimensions.NEXT_MEASUREMENT),
                                                                            (Dimensions.NEXT_MEASUREMENT, Dimensions.STATE_INPUT)]:
                                                    for state_activation in ["tanh", "ltanh", None]:
                                                        state_regularizer = None
                                                        if state_activation is None:
                                                            state_regularizer = "mse"

                                                        
                                                        configurations[n_variation] = ((n_segment, fast_forward, stable_state_epochs, check_input_masks, state_save_method, mask_sensorial_data, 
                                                                            short_time_recall, recall_stride, recall_n_past, recall_n_future,
                                                                            positional_encoder_type, sensorial_train_losses, block_configuration, state_activation ))
                                                        
                                                        n_variation += 1

n_variation, n_variation/800, (n_variation/800)**0.5

(1152, 1.44, 1.2)

In [59]:
(30*7*24*60*60)/22757.4

797.2791267895277

In [60]:
n_variation/((30*7*24*60*60)/22757.1333333)

11.559178835961905

In [45]:
with open("test.bin", "wb") as file:
    pickle.dump(configurations, file)

In [46]:
with open("test.bin", "rb") as file:
    configurations2 = pickle.load(file)

In [48]:
configurations2[0]

(1,
 False,
 1,
 True,
 <StateSaveMethod.MEAN: 1>,
 0.5,
 (<Dimensions.NEXT_MEASUREMENT: 2>,),
 1,
 0,
 1,
 'alibi',
 (),
 (<Dimensions.NEXT_MEASUREMENT: 2>, <Dimensions.NEXT_MEASUREMENT: 2>),
 'tanh')